In [1]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI

In [2]:
graph = Neo4jGraph(
    url="bolt://localhost:7687", username="bjm9889", password="12341234aB"
)

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

In [4]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [5]:
from langchain.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:
# 인천캠퍼스의 주소가 뭐야??
MATCH (campus:Campus {{name: '인천캠퍼스'}})-[:LOCATED_AT]->(address:Address) 
RETURN address.name

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [7]:
chain.run("홍성캠퍼스의 주소가 뭐야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (campus:Campus {name: '홍성캠퍼스'})-[:LOCATED_AT]->(address:Address) 
RETURN address.name
Full Context:
[{'address.name': '충남 홍성군 홍성읍 대학1길 66'}]

> Finished chain.


'홍성캠퍼스의 주소는 충남 홍성군 홍성읍 대학1길 66입니다.'

In [6]:
chain.run("인천캠퍼스의 주소가 뭐야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (campus:Campus {name: '인천캠퍼스'})-[:LOCATED_AT]->(address:Address) 
RETURN address.name
Full Context:
[{'address.name': '인천광역시 미추홀구 숙골로 113(도화동)'}]

> Finished chain.


'인천캠퍼스의 주소는 인천광역시 미추홀구 숙골로 113(도화동)입니다.'

In [6]:
chain.run("컴퓨터공학과는 어떤 학과야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (department:Department {name: '컴퓨터공학과'})-[:HAS_FACULTY]->(faculty:Faculty) 
RETURN faculty.name
Full Context:
[{'faculty.name': '컴퓨터공학과 교수진'}]

> Finished chain.


'컴퓨터공학과는 컴퓨터 공학 분야에서 교육과 연구를 수행하는 학과입니다. 이 학과의 교수진은 컴퓨터 공학 분야에서 전문적인 지식과 경험을 가지고 있습니다.'

In [8]:
chain.run("학부생의 정기주차 요금은 얼마야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (subscriptionParking:SubscriptionParking)-[:HAS_UNDERGRADUATE_FEE]->(subscriptionFee:SubscriptionFee)
RETURN subscriptionFee.name
Full Context:
[{'subscriptionFee.name': '5000원/월 혹은 20,000원/학기'}]

> Finished chain.


'학부생의 정기주차 요금은 5000원/월 혹은 20,000원/학기입니다.'

In [9]:
chain.run("청운대학교 eclass 웹페이지 주소가 뭐야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (university:University {name: '청운대학교'})-[:OPERATES]->(cwuwebpage:Cwuwebpage)-[:OFFERS_EClass]->(webpage:Webpage)
RETURN webpage.name
Full Context:
[{'webpage.name': 'https://cyber.chungwoon.ac.kr/ilos/main/main_form.acl'}]

> Finished chain.


'청운대학교 eclass 웹페이지 주소는 https://cyber.chungwoon.ac.kr/ilos/main/main_form.acl 입니다.'

In [6]:
chain.run("인천캠퍼스의 주소와 홍성캠퍼스의 주소가 뭐야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (campus:Campus {name: '인천캠퍼스'})-[:LOCATED_AT]->(address1:Address), (campus2:Campus {name: '홍성캠퍼스'})-[:LOCATED_AT]->(address2:Address) 
RETURN address1.name, address2.name
Full Context:
[{'address1.name': '인천광역시 미추홀구 숙골로 113(도화동)', 'address2.name': '충남 홍성군 홍성읍 대학1길 66'}]

> Finished chain.


'인천캠퍼스의 주소는 인천광역시 미추홀구 숙골로 113(도화동)이고, 홍성캠퍼스의 주소는 충남 홍성군 홍성읍 대학1길 66입니다.'